## Analysis of CMIP5-historical dataset on Temperature and Precipitation
#### Prepared by Vignesh Sridharan 

In [ ]:
#libraries for Climate data analysis
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as colors
import matplotlib.cm as cm
import cartopy.crs as ccrs
import ipywidgets as widgets
import cdsapi
import pickle
%matplotlib inline

In [ ]:
# Downloading the CMIP5-historical Mean precipitation flux and Surface-air temperature (2m) between 1850 and 2005 
# commented out as the data is downloaded and used from local repository
'''
import cdsapi
c = cdsapi.Client()
c.retrieve(
    'projections-cmip5-monthly-single-levels',
    {
        'experiment': 'historical',
        'variable': [
            '2m_temperature', 'mean_precipitation_flux',
        ],
        'model': 'noresm1_m',
        'ensemble_member': 'r1i1p1',
        'period': '185001-200512',
        'format': 'zip',
    },
    'download.zip')
'''

In [ ]:
#Loading the *.nc file on historical precipitation and temperature between 1850 and 2005
#The filesize was large, hence the pickled-format of the files are used
ds1 = pickle.load( open( "./pr_mon_NorESM_historical_1850-2005.p", "rb" ) )# Precipitation
ds2 = pickle.load( open( "./tas_mon_NorESM_historical_1850-2005.p", "rb" ) )# Temperature
#Loading the *csv file with the 'lat' and 'lon' coordinates for all the capital cities in the world  
coord=pd.read_csv('./latlon.csv')

#### Unit adjustments for Temperature and Precipitation

In [ ]:
#converting the pecipitation data from SI untis to mm/month
ds1.pr.data = ds1.pr.data * 86400*30
ds1.pr.attrs['units'] = 'mm/month' 
#converting the Surface air temperature data from SI untis to deg Celcius
ds2.tas.data = ds2.tas.data-273.15
ds2.tas.attrs['units'] = 'deg C' 

In [ ]:
#Simple global plot to analyse the historical precipitation and temperature patterns 
def plot_global (timestamp):
    time=str(timestamp)# string conversation for use in plot title
    time=str(time.split(' ')[0])    
    f,((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(24, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    ds1['pr'].sel(time = timestamp).plot(ax=ax1,cmap='rainbow')
    ds1.pr.mean(dim='time').plot(ax=ax2, cmap='rainbow',cbar_kwargs={'label': 'Precipitation [mm/month]'})
    ds2['tas'].sel(time = timestamp).plot(ax=ax3,cmap='rainbow',vmin=-80,vmax= 60)
    ds2.tas.mean(dim='time').plot(ax=ax4,cmap='rainbow',vmin=-80,vmax= 60,cbar_kwargs={'label': 'Near-surface Air temperature \n [deg C]'},)
    ax1.set_title('Average monthly Precipitation on chosen time stamp' +'_'+ time,fontsize=15)
    ax2.set_title('Average monthly precipitation for period (1850-2005)',fontsize=15)
    ax3.set_title('Average monthly temperature on chosen time stamp' +'_'+ time,fontsize=15)
    ax4.set_title('Average monthly temperature for period (1850-2005)',fontsize=15)
    ax1.coastlines()
    ax2.coastlines()
    ax3.coastlines()
    ax4.coastlines()
    f.suptitle('Global variations in temperature and precipitation', fontsize=30,y=1.01)

#Simple line plot to analyse the historical precipitation and temperature patterns in all the capital cities of the world 
def plot_country_cap (country):
    capital=coord[coord['Country Name']==country]['Capital Name'].copy()
    capital=capital.to_string(index=False)
    cap_lat=coord[coord['Country Name']==country]['Capital Latitude'].tolist()
    cap_lon=coord[coord['Country Name']==country]['Capital Longitude'].tolist()
    f,axes = plt.subplots(ncols=2,figsize=(10,5))
    mean_global_tas=ds2.tas.median(dim=['lat', 'lon'])
    mean_global_tas=mean_global_tas.resample(time='1Y').mean()
    con_cap_tas= ds2.sel(lat=cap_lat, lon=cap_lon, method='nearest')
    con_cap_tas=con_cap_tas.resample(time='1Y').mean()
    con_cap_tas.tas.plot(ax=axes[0],label=str(capital))
    mean_global_tas.plot(ax=axes[0],label= 'global mean')
    axes[0].set_title('Comparison of Surface air temperature',fontsize=14)
    axes[0].set_ylabel('Yearly average \n Surface air temperature (deg C)',fontsize=11)
    mean_global_pr=ds1.pr.mean(dim=['lat', 'lon'])
    mean_global_pr=mean_global_pr.resample(time='1Y').sum()
    con_cap_pr= ds1.sel(lat=cap_lat, lon=cap_lon, method='nearest')
    con_cap_pr=con_cap_pr.resample(time='1Y').sum()
    con_cap_pr.pr.plot(ax=axes[1],label=str(capital))
    mean_global_pr.plot(ax=axes[1],label='global mean')
    axes[1].set_title('Comparison of Precipitation',fontsize=14)
    axes[1].set_ylabel('Yearly total Precipitation (mm)',fontsize=11)
    axes[0].legend(loc=0)
    axes[1].legend(loc=0)
    plt.tight_layout()
    f.suptitle('Variations in temperature and precipitation (' + capital + ' vs Global-mean)', fontsize=30,y=1.18)   


### The following widgets examine the temperature and precipitation variations across the globe

#### The comparison is done between monthly (historical) and averaged monthly data for the period: 1850-2005

In [ ]:
#simple widget to analyse the global variation in tenmperature and precipitation to compare with the time period pm
select_timestamp = widgets.Dropdown(
    options=ds1.time.values.tolist(),
    description='Timestamp:',
    disabled=False
     )
print ('Choose the time stamp (between 1850-2005) to analyse')
interact_plot = widgets.interact(plot_global, timestamp = select_timestamp)

### The following widget examines the temperature and precipitation variations across capital cities of the world

#### The comparison is done between the observations for the capital city and the global-mean

In [ ]:
# Widget to choose the country (capital)
select_country = widgets.Dropdown(
    options=coord['Country Name'].to_list(),
    description='Country:',
    disabled=False
    )
print ('Choose a country (capital) to analyse')
interact_plot = widgets.interact(plot_country_cap, country=select_country)